In [74]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import os

In [75]:
browser = webdriver.Chrome()
url = 'https://finance.naver.com/sise/sise_market_sum.naver?&page='

# 브라우저 최대화
# browser.maximize_window()

# 사이트 이동
browser.get(url)

In [76]:
# 조회 항목 초기화
checkboxes = browser.find_elements(By.NAME,'fieldIds')
for checkbox in checkboxes:
    if checkbox.is_selected(): # 선택이 되었다면
        checkbox.click()
        
# 3. 조회 항목 설정 (원하는 항목)
items_to_select = ['영업이익','자산총계','매출액']
# 실제로 체크할 항목을 보면 영업이익(억)과 같이 적혀있지만 (억)은 별도로 적혀 있음 
# html 구성상을 보고 판단해야함 .

# checkbox
for checkbox in checkboxes:
    parent = checkbox.find_element(By.XPATH,'..') # 부모 엘리먼트 td를 찾게 됨
    label = parent.find_element(By.TAG_NAME,'label') # td 엘리먼트 안에 label 엘리먼트를 찾음
    # print(label.text) # 이름확인하기 위해 하는것
    if label.text in items_to_select: # 선택항목과 일치한다면
        checkbox.click() # 항목 체크 
    

In [77]:
# 적용하기 버튼 클릭
# 방법 1 xpath 로 구하기
# # 네이버 메인화면 -> 개발자 도구 -> 누를 버튼을 개발자도구 태그 찾는걸로 누르고
# # href 있는 태그를 우클릭해서 xpath 복사하기를 한다.
# btn_apply = browser.find_element(By.XPATH,'//*[@id="contentarea_left"]/div[2]/form/div/div/div/a[1]')
# btn_apply.click()

# 방법 2 href로 찾기
# <a href="javascript:fieldSubmit()"> 와 같이 자바스크립트로 명시되어 있으므로
# 시작에 // => 전체 html 문서에서 찾겠다

btn_apply = browser.find_element(By.XPATH,'//a[@href="javascript:fieldSubmit()"]')
btn_apply.click()

In [78]:
# 데이터 추출
# 판다스의 read_html 을 이용해 편하게 가져오기
df = pd.read_html(browser.page_source)


In [79]:
len(df) # df가 몇개의 테이블을 가져오나 확인

3

In [80]:
df[0] # 확인해보면, 옵션 선택하는 부분임 => 이부분은 우리가 원하는 부분이 아님

,0,1,2,3,4,5
0,거래량,매수호가,거래대금(백만),시가총액(억),영업이익(억),PER(배)
1,시가,매도호가,전일거래량,자산총계(억),영업이익증가율,ROE(%)
2,고가,매수총잔량,외국인비율,부채총계(억),당기순이익(억),ROA(%)
3,저가,매도총잔량,상장주식수(천주),매출액(억),주당순이익(원),PBR(배)
4,NaN,NaN,NaN,매출액증가율,보통주배당금(원),유보율(%)


In [81]:
df[1] # 우리가 원하는 정보임

,N,종목명,현재가,전일비,등락률,액면가,매출액,자산총계,영업이익,토론실,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,삼성전자,71000.0,1600.0,-2.20%,100.0,3022314.0,4484245.0,433766.0,NaN,NaN,NaN,NaN
2,2.0,SK하이닉스,131000.0,1100.0,-0.83%,5000.0,446216.0,1038715.0,68094.0,NaN,NaN,NaN,NaN
3,3.0,LG에너지솔루션,390000.0,10500.0,-2.62%,500.0,255986.0,382994.0,12137.0,NaN,NaN,NaN,NaN
4,4.0,삼성바이오로직스,756000.0,10000.0,-1.31%,2500.0,30013.0,165821.0,9836.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,49.0,TIGER CD금리투자KIS(합성),53670.0,5.0,+0.01%,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,50.0,아모레퍼시픽,121300.0,1500.0,-1.22%,500.0,41349.0,58018.0,2142.0,NaN,NaN,NaN,NaN
78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
df = pd.read_html(browser.page_source)[1] # 필요한 데이터만 가져옴
df

,N,종목명,현재가,전일비,등락률,액면가,매출액,자산총계,영업이익,토론실,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,삼성전자,71000.0,1600.0,-2.20%,100.0,3022314.0,4484245.0,433766.0,NaN,NaN,NaN,NaN
2,2.0,SK하이닉스,131000.0,1100.0,-0.83%,5000.0,446216.0,1038715.0,68094.0,NaN,NaN,NaN,NaN
3,3.0,LG에너지솔루션,390000.0,10500.0,-2.62%,500.0,255986.0,382994.0,12137.0,NaN,NaN,NaN,NaN
4,4.0,삼성바이오로직스,756000.0,10000.0,-1.31%,2500.0,30013.0,165821.0,9836.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,49.0,TIGER CD금리투자KIS(합성),53670.0,5.0,+0.01%,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,50.0,아모레퍼시픽,121300.0,1500.0,-1.22%,500.0,41349.0,58018.0,2142.0,NaN,NaN,NaN,NaN
78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
df.head(20) 
# 데이터를 확인해보면 처음에 nan 있고 중간중간에 nan이 3줄씩 있음
# 데이터를 정제해줘야함

,N,종목명,현재가,전일비,등락률,액면가,매출액,자산총계,영업이익,토론실,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,삼성전자,71000.0,1600.0,-2.20%,100.0,3022314.0,4484245.0,433766.0,NaN,NaN,NaN,NaN
2,2.0,SK하이닉스,131000.0,1100.0,-0.83%,5000.0,446216.0,1038715.0,68094.0,NaN,NaN,NaN,NaN
3,3.0,LG에너지솔루션,390000.0,10500.0,-2.62%,500.0,255986.0,382994.0,12137.0,NaN,NaN,NaN,NaN
4,4.0,삼성바이오로직스,756000.0,10000.0,-1.31%,2500.0,30013.0,165821.0,9836.0,NaN,NaN,NaN,NaN
5,5.0,삼성전자우,58300.0,800.0,-1.35%,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6.0,셀트리온,178000.0,9500.0,-5.07%,1000.0,22840.0,58917.0,6472.0,NaN,NaN,NaN,NaN


In [84]:
df.dropna(axis=0,how='all',inplace=True)
# 행기준으로 삭제, 
# how = 'all' => 모든 행이 NaN이면 삭제, any면 하나라도 NaN이면 삭제

In [85]:
df 
# 최대 6개까지 데이터를 갖고 있다고 생각했는데 3개만 골라서 3개는 빈 테이블이 있음
# 이건 네이버 개발자가 이렇게 코드를 짜놔서 그럼. 그리고 토론실도 그냥 이동하는거라 비어있음
# 따라서 이것도 데이터 정제를 해줘야함

,N,종목명,현재가,전일비,등락률,액면가,매출액,자산총계,영업이익,토론실,Unnamed: 10,Unnamed: 11,Unnamed: 12
1,1.0,삼성전자,71000.0,1600.0,-2.20%,100.0,3022314.0,4484245.0,433766.0,NaN,NaN,NaN,NaN
2,2.0,SK하이닉스,131000.0,1100.0,-0.83%,5000.0,446216.0,1038715.0,68094.0,NaN,NaN,NaN,NaN
3,3.0,LG에너지솔루션,390000.0,10500.0,-2.62%,500.0,255986.0,382994.0,12137.0,NaN,NaN,NaN,NaN
4,4.0,삼성바이오로직스,756000.0,10000.0,-1.31%,2500.0,30013.0,165821.0,9836.0,NaN,NaN,NaN,NaN
5,5.0,삼성전자우,58300.0,800.0,-1.35%,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6.0,셀트리온,178000.0,9500.0,-5.07%,1000.0,22840.0,58917.0,6472.0,NaN,NaN,NaN,NaN
10,7.0,현대차,181800.0,4400.0,-2.36%,5000.0,1425275.0,2557425.0,98198.0,NaN,NaN,NaN,NaN
11,8.0,NAVER,219000.0,11000.0,-4.78%,100.0,82201.0,338990.0,13047.0,NaN,NaN,NaN,NaN
12,9.0,POSCO홀딩스,419000.0,18500.0,-4.23%,5000.0,847502.0,984068.0,48501.0,NaN,NaN,NaN,NaN
13,10.0,기아,87900.0,1900.0,-2.12%,5000.0,865590.0,737110.0,72331.0,NaN,NaN,NaN,NaN


In [86]:
df.dropna(axis=1,how='all',inplace=True)
df

,N,종목명,현재가,전일비,등락률,액면가,매출액,자산총계,영업이익
1,1.0,삼성전자,71000.0,1600.0,-2.20%,100.0,3022314.0,4484245.0,433766.0
2,2.0,SK하이닉스,131000.0,1100.0,-0.83%,5000.0,446216.0,1038715.0,68094.0
3,3.0,LG에너지솔루션,390000.0,10500.0,-2.62%,500.0,255986.0,382994.0,12137.0
4,4.0,삼성바이오로직스,756000.0,10000.0,-1.31%,2500.0,30013.0,165821.0,9836.0
5,5.0,삼성전자우,58300.0,800.0,-1.35%,100.0,NaN,NaN,NaN
9,6.0,셀트리온,178000.0,9500.0,-5.07%,1000.0,22840.0,58917.0,6472.0
10,7.0,현대차,181800.0,4400.0,-2.36%,5000.0,1425275.0,2557425.0,98198.0
11,8.0,NAVER,219000.0,11000.0,-4.78%,100.0,82201.0,338990.0,13047.0
12,9.0,POSCO홀딩스,419000.0,18500.0,-4.23%,5000.0,847502.0,984068.0,48501.0
13,10.0,기아,87900.0,1900.0,-2.12%,5000.0,865590.0,737110.0,72331.0


In [87]:
# 데이터를 처음에 받아올때는 위에 종목명 현재가 같은 헤더 부분을 가져오고
# 2페이지부터는 헤더를 제외하고 가져오면 됨

# 파일 저장
# f_name = 'sise.csv'
# if os.path.exists(f_name): #파일이 있다면? 헤더 제외
#     df.to_csv(f_name,encoding='utf-8-sig',index=False,mode='a',header=False)
#     # mode = 'a' => append 뒤에 추가해라라는 의미
# else : # 파일이 없다면
#     df.to_csv(f_name,encoding='utf-8-sig',index=False)
#     # 아무것도 안적어주면 쓰기모드에 헤더를 넣어줌
# print(f'{idx} 페이지 완료')



In [88]:
# 실제로 반복을 돌 부분
f_name = 'sise.csv'

for idx in range(1,40):
    # 사전 작업 : 페이지 이동
    browser.get(url + str(idx))
    
    df = pd.read_html(browser.page_source)[1]
    df.dropna(axis=0,how='all',inplace=True)
    df.dropna(axis=1,how='all',inplace=True)
    
     # 만약 df가 비어있으면 반복문 탈출
    if len(df) == 0:
        break
    
    
    
    # 파일 저장
    if os.path.exists(f_name): #파일이 있다면? 헤더 제외
        df.to_csv(f_name,encoding='utf-8-sig',index=False,mode='a',header=False)
        # mode = 'a' => append 뒤에 추가해라라는 의미
    else : # 파일이 없다면
        df.to_csv(f_name,encoding='utf-8-sig',index=False)
        #아무것도 안적어주면 쓰기모드에 헤더를 넣어줌  
        print(f'{idx} 페이지 완료')

browser.close()

1 페이지 완료
